In [13]:
import csv
import time
import random
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementNotInteractableException,
    ElementClickInterceptedException
)
from dotenv import load_dotenv
from selenium.webdriver.chrome.options import Options

In [14]:
load_dotenv()

CHATGPT_API_KEY = os.getenv("OPENAI_KEY")
CHATGPT_BASE_URL = os.getenv("CHATGPT_BASE_URL")

In [15]:
# A list of sentences to end a conversation
farewell_messages = [
    "I think it's time for me to head out now.",
    "It seems like a good time to wrap things up.",
    "I probably should get going now.",
    "Let's go ahead and call it a day for now.",
    "I think I'm going to log off now.",
    "I'm gonna bounce out of this convo.",
    "I think I'm done chatting for now.",
    "I'll just leave things here for now.",
    "I'm going to go ahead and sign off.",
    "I'll catch you later; thanks for the chat!",
    "I'm stepping out for now.",
    "I think I'm going to tap out here.",
    "I'm just gonna close this chat now.",
    "That's enough chatting for me today.",
    "I'll stop here, thanks for everything!",
    "I'm going to head off now.",
    "I'll catch you next time we chat.",
    "That'll be it for me today!",
    "I'm good for now; thanks a lot!",
    "Wrapping things up here; talk soon!",
    "That's all from my end for now.",
    "I think I'll call it quits here.",
    "I'll go ahead and check out of this chat.",
    "I'm about to take off now.",
    "Guess I'll go ahead and head out.",
    "I think I'll dip out of the convo now.",
    "I'm going to go ahead and sign out.",
    "All set on my end, thanks a lot!",
    "I'll take a break from this chat.",
    "I think I'll log off now.",
    "I'm just going to step away for now.",
    "I'll catch you on the flip side!",
    "I think I'll go ahead and move on.",
    "Alright, it's time for me to peace out.",
    "This is where I'll wrap things up.",
    "I'll just leave it here for now.",
    "I'll go ahead and close out this chat.",
    "Heading out of the convo; thanks!",
    "I'm checking out of this conversation.",
    "I'll go ahead and bounce for now.",
    "I'm going offline now.",
    "Ready to wrap things up here.",
    "I'm going offline at this point.",
    "Thanks a lot; I think I'm all set now.",
    "I'm going to leave the chat for now.",
    "I'm good to wrap things up here.",
    "I'll go ahead and close out now.",
    "I'll go ahead and sign off here.",
    "I'm going to wrap up on my end.",
    "I'll be taking a break here. Thanks!"
]

In [16]:
def send_message_to_chatgpt(conversation_history):
    time.sleep(20)
    headers = {
        "Authorization": f"Bearer {CHATGPT_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "gpt-4o",
        "messages": conversation_history,
        "max_tokens": 150,
        "temperature": 0.7
    }

    try:
        response = requests.post(CHATGPT_BASE_URL, json=payload, headers=headers, timeout=30)
        response.raise_for_status()
        response_json = response.json()
        return response_json['choices'][0]['message']['content']
    except requests.RequestException as e:
        print(f"Error connecting to ChatGPT API: {e}")
        return "Sorry, I couldn't generate a response right now."

In [17]:
def wait_for_user_login(driver):
    print("Please log in to JuicyChat and press Enter to continue.")
    input("Press Enter after you have logged in successfully: ")

In [18]:
def select_random_character(driver):
    try:
        character_cards = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.juicyHomeCharacterCard"))
        )
        if not character_cards:
            raise Exception("No character cards found!")

        random_card = random.choice(character_cards)
        random_card_link = random_card.find_element(By.CSS_SELECTOR, "a._cardATag_1oit3_107")
        driver.execute_script("arguments[0].click();", random_card_link)

        print("Random character selected. Proceeding to chat...")
    except Exception as e:
        print(f"Error selecting random character: {str(e)}")
        raise

In [19]:
def get_latest_response(driver):
    try:
        response_elements = driver.find_elements(By.CSS_SELECTOR, "div._otherChatDetail_1nal2_10 div._markdownContent_1nal2_178")
        if response_elements:
            latest_response = response_elements[-1].text.strip()
            return latest_response
        else:
            return "No response received."
    except Exception as e:
        print(f"Error retrieving chat responses: {str(e)}")
        return "No response received."

In [20]:
def click_return_home_button(driver):
    try:
        home_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "div._logo_mozab_59"))
        )
        driver.execute_script("arguments[0].click();", home_button)
        print("Returned to homepage.")
    except Exception as e:
        print(f"Error clicking the return home button: {str(e)}")
    try:
        home_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div._logo_mozab_59 img"))
        )
        driver.execute_script("arguments[0].click();", home_button)
        print("Returned to homepage.")
    except Exception as e:
        print(f"Error clicking the return home button: {str(e)}")
        print("Returned to homepage.")
    except Exception as e:
        print(f"Error clicking the return home button: {str(e)}")

In [21]:
def send_message_and_wait_for_response(driver, message):
    try:
        chat_box = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "textarea[placeholder='Enter to send text. Option+Enter for linebreak.']"))
        )
        chat_box.click()
        chat_box.clear()
        chat_box.send_keys(message)

        send_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "div[class*='_sendBtn']"))
        )
        driver.execute_script("arguments[0].click();", send_button)

        time.sleep(10)
        latest_response = get_latest_response(driver)
        return latest_response
    except Exception as e:
        print(f"Error sending message and waiting for response: {str(e)}")
        return "No response received."

In [22]:
def conduct_conversation(writer):
    driver = None
    try:
        driver = webdriver.Firefox()
        driver.get("https://www.juicychat.ai/")
        wait_for_user_login(driver)

        for _ in range(9):
            select_random_character(driver)

            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "textarea[placeholder='Enter to send text. Option+Enter for linebreak.']"))
            )

            conversation_log = [None] * 10

            for i in range(4):
                if i == 0:
                    prompt = "Start a conversation with JuicyChat by asking a question or making a casual comment."
                else:
                    latest_response = conversation_log[2 * i - 1] if conversation_log[2 * i - 1] else "No prior responses."
                    prompt = f"JuicyChat just said: '{latest_response}'. Respond naturally."

                chatgpt_messages = [{"role": "system", "content": prompt}]
                user_message = send_message_to_chatgpt(chatgpt_messages)

                juicy_responses = send_message_and_wait_for_response(driver, user_message)
                juicy_response = juicy_responses if juicy_responses else "No response received."

                conversation_log[2 * i] = user_message
                conversation_log[2 * i + 1] = juicy_response

            time.sleep(20)
            farewell_message = random.choice(farewell_messages)
            juicy_responses = send_message_and_wait_for_response(driver, farewell_message)
            bot_farewell = juicy_responses if juicy_responses else "No response received."
            conversation_log[8] = farewell_message
            conversation_log[9] = bot_farewell

            writer.writerow(conversation_log)
            print("Saved conversation to CSV.")
            
            time.sleep(25)
            click_return_home_button(driver)

    except Exception as e:
        print(f"Error during conversation: {str(e)}")
    finally:
        if driver:
            driver.quit()

In [23]:
def main():
    with open('JuicyChat_conversation_test.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_ALL)
        headers = [
            "GPT_Message_1", "JuicyChat_Response_1",
            "GPT_Message_2", "JuicyChat_Response_2",
            "GPT_Message_3", "JuicyChat_Response_3",
            "GPT_Message_4", "JuicyChat_Response_4",
            "Farewell_Message", "Bot_Farewell_Response"
        ]
        writer.writerow(headers)

        conduct_conversation(writer)

In [24]:
if __name__ == "__main__":
    main()

Please log in to JuicyChat and press Enter to continue.
Random character selected. Proceeding to chat...
Error during conversation: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

